In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("dataset/58606.23.06.2020.23.06.2025.1.0.0.cn.utf8.00000000.xls")

In [3]:
dd_map = {
    "从北方吹来的风": 0,
    "从东北偏北方向吹来的风": 1,
    "从东北方吹来的风": 2,
    "从东北偏东方向吹来的风": 3,
    "从东方吹来的风": 4,
    "从东南偏东方向吹来的风": 5,
    "从东南方吹来的风": 6,
    "从东南偏南方向吹来的风": 7,
    "从南方吹来的风": 8,
    "从西南偏南方向吹来的风": 9,
    "从西南方吹来的风": 10,
    "从西南偏西方向吹来的风": 11,
    "从西方吹来的风": 12,
    "从西北偏西方向吹来的风": 13,
    "从西北方吹来的风": 14,
    "从西北偏北方向吹来的风": 15,
    "无风": 16
}
df["DD"] = df["DD"].map(dd_map).fillna(df["DD"])


In [4]:
pooled_df = df.groupby(df.index // 5).mean(numeric_only=True).reset_index(drop=True)
